In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from obspy import read, UTCDateTime
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import pywt  # Using PyWavelets instead of deprecated scipy.signal.cwt

# Set random seed for reproducibility
np.random.seed(42)

# 1. Data Loading
def load_catalog(file_path):
    return pd.read_csv(file_path)

def load_seismic_data(file_path):
    return read(file_path)

# Load the catalog
cat_directory = './data/lunar/training/catalogs/'
cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
catalog = load_catalog(cat_file)

print("Catalog loaded. Shape:", catalog.shape)
print(catalog.head())

# 2. Data Preprocessing
def preprocess_seismic_data(stream, start_time, end_time):
    start_time = UTCDateTime(start_time.isoformat())
    end_time = UTCDateTime(end_time.isoformat())
    
    stream = stream.trim(starttime=start_time, endtime=end_time)
    stream.detrend('linear')
    stream.filter('bandpass', freqmin=0.1, freqmax=3.3125)  # Adjusted to be below Nyquist
    return stream

# 3. Advanced Feature Extraction
def extract_features(trace):
    data = trace.data
    mean = np.mean(data)
    std = np.std(data)
    max_amp = np.max(np.abs(data))
    rms = np.sqrt(np.mean(data**2))
    kurtosis = np.mean((data - mean)**4) / std**4
    
    fft = np.fft.fft(data)
    freq = np.fft.fftfreq(len(data), d=trace.stats.delta)
    power = np.abs(fft)**2
    dominant_freq = freq[np.argmax(power)]
    spectral_centroid = np.sum(freq * power) / np.sum(power)
    
    # Using PyWavelets for wavelet transform
    widths = np.arange(1, 128)
    coeffs = pywt.cwt(data, 'cmor1.5-1.0', widths)[0]
    wavelet_energy = np.sum(np.abs(coeffs)**2, axis=0)
    wavelet_mean = np.mean(wavelet_energy)
    wavelet_std = np.std(wavelet_energy)
    
    return [mean, std, max_amp, rms, kurtosis, dominant_freq, spectral_centroid, wavelet_mean, wavelet_std]

# 4. Prepare Dataset
def prepare_dataset(catalog, data_directory, window_size=600):
    X = []
    y = []
    
    for _, row in catalog.iterrows():
        file_name = row['filename']
        event_time = pd.to_datetime(row['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])
        
        mseed_file = os.path.join(data_directory, file_name + '.mseed')
        print("Loading file:", mseed_file)  # Debugging line
        
        if not os.path.isfile(mseed_file):
            print(f"File not found: {mseed_file}. Skipping this entry.")
            continue
        
        stream = load_seismic_data(mseed_file)
        
        # Event window
        start_time = event_time - pd.Timedelta(seconds=window_size//2)
        end_time = event_time + pd.Timedelta(seconds=window_size//2)
        processed_stream = preprocess_seismic_data(stream, start_time, end_time)
        features = extract_features(processed_stream[0])
        X.append(features)
        y.append(1)  # 1 for event
        
        # Non-event window
        non_event_start = start_time - pd.Timedelta(seconds=window_size)
        non_event_end = start_time
        non_event_stream = preprocess_seismic_data(stream, non_event_start, non_event_end)
        non_event_features = extract_features(non_event_stream[0])
        X.append(non_event_features)
        y.append(0)  # 0 for non-event
    
    return np.array(X), np.array(y)

# Prepare the dataset
data_directory = './data/lunar/training/data/S12_GradeA/'
X, y = prepare_dataset(catalog, data_directory)

print("Dataset prepared. Shape:", X.shape)

# 5. Exploratory Data Analysis
def plot_feature_distributions(X, y):
    feature_names = ['Mean', 'Std Dev', 'Max Amplitude', 'RMS', 'Kurtosis', 
                     'Dominant Frequency', 'Spectral Centroid', 'Wavelet Mean', 'Wavelet Std']
    df = pd.DataFrame(X, columns=feature_names)
    df['Event'] = y
    
    fig, axes = plt.subplots(3, 3, figsize=(20, 20))
    axes = axes.ravel()
    
    for i, feature in enumerate(feature_names):
        sns.histplot(data=df, x=feature, hue='Event', kde=True, ax=axes[i])
        axes[i].set_title(feature)
    
    plt.tight_layout()
    plt.show()

plot_feature_distributions(X, y)

# 6. Model Preparation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# 7. Build and Train Model
def build_model(input_shape):
    model = Sequential([
        Conv1D(64, 3, activation='relu', input_shape=input_shape),
        MaxPooling1D(2),
        Conv1D(128, 3, activation='relu'),
        MaxPooling1D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = build_model((X_train_reshaped.shape[1], 1))
print(model.summary())

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_reshaped, y_train, 
                    epochs=100, 
                    batch_size=32, 
                    validation_split=0.2,
                    callbacks=[early_stopping],
                    verbose=1)

# 8. Model Evaluation
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

evaluate_model(model, X_test_reshaped, y_test)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# 9. Feature Importance
def plot_feature_importance(model, feature_names):
    # For CNN, we'll use the weights of the first convolutional layer
    weights = model.layers[0].get_weights()[0]
    importance = np.sum(np.abs(weights), axis=(0, 1))
    
    plt.figure(figsize=(10, 6))
    plt.bar(feature_names, importance)
    plt.title("Feature Importance")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

feature_names = ['Mean', 'Std Dev', 'Max Amplitude', 'RMS', 'Kurtosis', 
                 'Dominant Frequency', 'Spectral Centroid', 'Wavelet Mean', 'Wavelet Std']
plot_feature_importance(model, feature_names)

# 10. Prediction on New Data
def predict_event(model, scaler, file_path, event_time, window_size=600):
    stream = load_seismic_data(file_path)
    start_time = event_time - pd.Timedelta(seconds=window_size//2)
    end_time = event_time + pd.Timedelta(seconds=window_size//2)
    processed_stream = preprocess_seismic_data(stream, start_time, end_time)
    
    features = extract_features(processed_stream[0])
    features_scaled = scaler.transform([features])
    features_reshaped = features_scaled.reshape(features_scaled.shape[0], features_scaled.shape[1], 1)
    
    prediction = model.predict(features_reshaped)
    return prediction[0][0]

# Example usage
# file_path = './data/lunar/training/data/S12_GradeA/example.mseed'
# event_time = pd.to_datetime('1970-01-19T20:25:00.000000')
# prediction = predict_event(model, scaler, file_path, event_time)
# print(f"Prediction for event: {'Event' if prediction > 0.5 else 'Non-Event'}")


Catalog loaded. Shape: (76, 5)
                                 filename time_abs(%Y-%m-%dT%H:%M:%S.%f)  \
0  xa.s12.00.mhz.1970-01-19HR00_evid00002     1970-01-19T20:25:00.000000   
1  xa.s12.00.mhz.1970-03-25HR00_evid00003     1970-03-25T03:32:00.000000   
2  xa.s12.00.mhz.1970-03-26HR00_evid00004     1970-03-26T20:17:00.000000   
3  xa.s12.00.mhz.1970-04-25HR00_evid00006     1970-04-25T01:14:00.000000   
4  xa.s12.00.mhz.1970-04-26HR00_evid00007     1970-04-26T14:29:00.000000   

   time_rel(sec)       evid    mq_type  
0        73500.0  evid00002  impact_mq  
1        12720.0  evid00003  impact_mq  
2        73020.0  evid00004  impact_mq  
3         4440.0  evid00006  impact_mq  
4        52140.0  evid00007    deep_mq  
Loading file: ./data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-01-19HR00_evid00002.mseed


C:\Users\dipes\.conda\envs\project\Lib\site-packages\obspy\signal\filter.py:62: UserWarning: Selected high corner frequency (3.3125) of bandpass is at or above Nyquist (3.3125). Applying a high-pass instead.
  warnings.warn(msg)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()